In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import modules 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Reshape, Conv2D, Concatenate, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D, SpatialDropout1D, MaxPool2D, BatchNormalization, Wrapper, InputSpec, TimeDistributed, concatenate
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split, KFold, StratifiedShuffleSplit, StratifiedKFold
from sklearn.metrics import (
      accuracy_score,
      recall_score,
      precision_score,
      f1_score, roc_curve, auc)
from keras.callbacks import EarlyStopping, ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=4,restore_best_weights=True,verbose=0)
callbacks=[early_stopping]
from keras import backend as K
import tensorflow as tf
import keras
import pandas as pd
from tensorflow.keras.optimizers import Adam
from gensim.models import KeyedVectors
import string

# Load data

In [ ]:
!git clone https://github.com/anonymoususr12/MHPurf

In [ ]:
%cd MHPurf/data
project = 'tensorflow' # select in [tensorflow, pytorch, keras, incubator-mxnet, caffe]
path = f'{project}.csv'
pd_all = pd.read_csv(path)
pd_all = pd_all.sample(frac=1,random_state=999)
pd_all['Title+Body'] = ''
for idx in range(len(pd_all)):
  if pd.notna(pd_all['Body'].iloc[idx]):
    pd_all['Title+Body'].iloc[idx] = pd_all['Title'].iloc[idx] + '. ' + pd_all['Body'].iloc[idx]
  else:
    pd_all['Title+Body'].iloc[idx] = pd_all['Title'].iloc[idx]
pd_title = pd_all.rename(columns={"Unnamed: 0":"id","class":"sentiment","Title":"text"})
pd_title.to_csv('Title.csv', index=False, columns=["id","Number","sentiment","text"], mode="w")
pd_body = pd_all.rename(columns={"Unnamed: 0":"id","class":"sentiment","Body":"text"})
pd_body.to_csv('Body.csv', index=False, columns=["id","Number","sentiment","text"], mode="w")
pd_label = pd_all.rename(columns={"Unnamed: 0":"id","class":"sentiment","Labels":"text"})
pd_label.to_csv('Labels.csv', index=False, columns=["id","Number","sentiment","text"], mode="w")
pd_code = pd_all.rename(columns={"Unnamed: 0":"id","class":"sentiment","Codes":"text"})
pd_code.to_csv('Codes.csv', index=False, columns=["id","Number","sentiment","text"], mode="w")
pd_comment = pd_all.rename(columns={"Unnamed: 0":"id","class":"sentiment","Comments":"text"})
pd_comment.to_csv('Comments.csv', index=False, columns=["id","Number","sentiment","text"], mode="w")
pd_command = pd_all.rename(columns={"Unnamed: 0":"id","class":"sentiment","Commands":"text"})
pd_command.to_csv('Command.csv', index=False, columns=["id","Number","sentiment","text"], mode="w")
pd_tplusb = pd_all.rename(columns={"Unnamed: 0":"id","class":"sentiment","Title+Body":"text"})
pd_tplusb.to_csv('Title+Body.csv', index=False, columns=["id","Number","sentiment","text"], mode="w")

# Training

In [ ]:
##################### GLOBAL ###################
NETWORK = 'LSTM' # choose LSTM or CNN

REPEAT = 30
repeated_range = range(0,REPEAT)

max_features = 1000
MAX_LEN = 250

embed_size = 100 # how big is each word vector
CV_TIME = 10

out_csv_name = f'../{project}_{NETWORK}'

if NETWORK == 'LSTM':
  out_csv_name += '_LSTM'
elif NETWORK == 'CNN':
  out_csv_name += '_CNN'

data = pd.read_csv('Title+Body.csv')
data = data.rename(columns={"sentiment": "target"})
#################################################
for i in range(len(data['text'])):
    data['text'].iloc[i] = str(data['text'].iloc[i])

# Preprocessing

import re

# removing URLs

def remove_url(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

data["text"] = data["text"].apply(lambda x: remove_url(x))

# removing html

def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(r'', text)

data["text"] = data["text"].apply(lambda x: remove_html(x))

# removing emoji

def remove_emoji(text):
    emoji_pattern = re.compile("["
                              u"\U0001F600-\U0001F64F" #emoticons
                              u"\U0001F300-\U0001F5FF" #symbols&pics
                              u"\U0001F680-\U0001F6FF" #transportation pic
                              u"\U0001F1E0-\U0001F1FF" #flags
                              u"\U00002702-\U000027B0"
                              u"\U000024C2-\U0001F251"    
                              "]+", flags = re.UNICODE)
    return emoji_pattern.sub(r'', text)

data["text"] = data["text"].apply(lambda x: remove_emoji(x))

# Stop Word Removal
NLTK_stop_words_list = stopwords.words('english')
custom_stop_words_list = ['...']
final_stop_words_list = NLTK_stop_words_list + custom_stop_words_list
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in final_stop_words_list])

data["text"] = data["text"].apply(lambda text: remove_stopwords(text))

# Symbol removal
def clean_str(string):
    string = re.sub(r"[^A-Za-z0-9(),.!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r"\\", "", string)
    string = re.sub(r"\'", "", string)
    string = re.sub(r"\"", "", string)
    return string.strip().lower()

data["text"] = data["text"].apply(lambda text: clean_str(text))

# Word2Vec Embedding
list_sentences = data["text"].fillna("").values
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences))
list_tokenized = tokenizer.texts_to_sequences(list_sentences)
y = data["target"]
data = pad_sequences(list_tokenized, maxlen=MAX_LEN)

def load_word2vec_embeddings(filepath, tokenizer, max_features, embedding_size):
    model = KeyedVectors.load_word2vec_format(filepath,limit=500000)

    emb_mean, emb_std = model.wv.syn0.mean(), model.wv.syn0.std()

    word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embedding_size))
    for word, i in word_index.items():
        if i > max_features:
            continue
        try:
            embedding_vector = model[word]
            embedding_matrix[i-1] = embedding_vector
        except KeyError:
            continue
    return embedding_matrix

# embedding_matrix = load_word2vec_embeddings("/content/drive/MyDrive/data_only/embedding/enwiki_20180420_100d.txt.bz2",
#                                             tokenizer,
#                                             max_features,
#                                             embed_size)
embedding_matrix = load_word2vec_embeddings("embedding/enwiki_20180420_100d.txt.bz2",
                                            tokenizer,
                                            max_features,
                                            embed_size)

# Model Construction
def build_model(max_len):
    if NETWORK == 'LSTM':
      # LSTM begin
      inp = Input(shape=(MAX_LEN,))
      x = Dropout(0.1)(inp)
      x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
      x = LSTM(100)(x)
      x = Dense(100, activation="relu")(x)
      x = Dropout(0.1)(x)
      x = Dense(2, activation="softmax")(x)
    
    if NETWORK == 'CNN':
      # cPur (CNN) begin
      filter_sizes = [3,4,5]
      num_filters = 2
      maxlen = MAX_LEN
      inp = Input(shape=(maxlen,))
      x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
      x = SpatialDropout1D(0.2)(x)
      x = Reshape((maxlen, embed_size, 1))(x)
      conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embed_size), kernel_initializer='normal',
                                                                                      activation='elu')(x)
      conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embed_size), kernel_initializer='normal',
                                                                                      activation='elu')(x)
      conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embed_size), kernel_initializer='normal',
                                                                                      activation='elu')(x)
      maxpool_0 = MaxPool2D(pool_size=(maxlen - filter_sizes[0] + 1, 1))(conv_0)
      maxpool_1 = MaxPool2D(pool_size=(maxlen - filter_sizes[1] + 1, 1))(conv_1)
      maxpool_2 = MaxPool2D(pool_size=(maxlen - filter_sizes[2] + 1, 1))(conv_2)
      z = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])   
      z = Flatten()(z)
      x = Dense(2, activation="softmax")(z)
    
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Training
macro = []
micro = []
auc_value = []
for repeated_time in repeated_range:
    cv_time = 1
    X_train, X_test, y_train, y_test = train_test_split(data,y,test_size=0.2, random_state=repeated_time)
    y_train = y_train.to_numpy()
    y_test = y_test.to_numpy()
  
    model = build_model(MAX_LEN)
    y_train = to_categorical(np.asarray(y_train))
    model.fit(X_train, y_train, batch_size=64, epochs=100, validation_split=0.1,callbacks=[early_stopping])
    y_pred = model.predict(X_test, batch_size=1024, verbose=1).round().astype(int)
    y_true = y_test
    y_true = to_categorical(np.asarray(y_true))
    current_macro=f1_score(y_true,y_pred,average='macro')
    macro.append(current_macro)
    current_micro=f1_score(y_true,y_pred,average='micro')
    micro.append(current_micro)
    fpr, tpr, thresholds = roc_curve(y_true[:,1], y_pred[:,1], pos_label=1)
    current_auc = auc(fpr, tpr)
    auc_value.append(current_auc)

    new_row = {'repeated_times':REPEAT,'cv_list':str(auc_value),'Macro F1':sum(macro) / len(macro), 'Micro F1':sum(micro) / len(micro), 'AUC':sum(auc_value) / len(auc_value)}
    df_log = pd.DataFrame(columns=['repeated_time','cv_list','Macro F1', 'Micro F1', 'AUC'])
    df_log = df_log.append(new_row, ignore_index=True)
    df_log.to_csv(out_csv_name, mode='a', header=False)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:98: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:98: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).


Epoch 1/100
17/17 [==============================] - 6s 43ms/step - loss: 0.5187 - accuracy: 0.7910 - val_loss: 0.4758 - val_accuracy: 0.8167
Epoch 2/100
17/17 [==============================] - 0s 16ms/step - loss: 0.4823 - accuracy: 0.8116 - val_loss: 0.4772 - val_accuracy: 0.8167
Epoch 3/100
17/17 [==============================] - 0s 16ms/step - loss: 0.4695 - accuracy: 0.8116 - val_loss: 0.4522 - val_accuracy: 0.8167
Epoch 4/100
17/17 [==============================] - 0s 16ms/step - loss: 0.4363 - accuracy: 0.8106 - val_loss: 0.4299 - val_accuracy: 0.8250
Epoch 5/100
17/17 [==============================] - 0s 16ms/step - loss: 0.3803 - accuracy: 0.8340 - val_loss: 0.4277 - val_accuracy: 0.8083
Epoch 6/100
17/17 [==============================] - 0s 16ms/step - loss: 0.3220 - accuracy: 0.8666 - val_loss: 0.4619 - val_accuracy: 0.7333
Epoch 7/100
17/17 [==============================] - 0s 16ms/step - loss: 0.2863 - accuracy: 0.8862 - val_loss: 0.4284 - val_accuracy: 0.8167
Epoch 